In [ ]:
pip install -r requirements.txt

In [33]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow
import sklearn
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  mean_absolute_error

In [6]:
base =pd.read_csv('petr4_treinamento.csv')  #base de dados

In [ ]:
base.isnull().sum() #verificar se tem valores nulos

In [8]:
base = base.dropna() #excluir valores nulos

In [9]:
base_treinamento = base.iloc[:, 1:2].values #variaveis independentes (preço de abertura)

In [10]:
normalizador = MinMaxScaler(feature_range=(0,1)) #normalizar os dados
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [11]:
X = [] 
y =[]
for i in range(90, 1242):
    X.append(base_treinamento_normalizada[i-90:i, 0]) #pegar os 90 primeiros valores e colocar em X
    y.append(base_treinamento_normalizada[i, 0]) #pegar o valor seguinte e colocar em y
X, y = np.array(X), np.array(y) #transformar em array

In [ ]:
regressor = Sequential() #iniciar a rede neural

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (X.shape[1], 1))) #camada de entrada com 100 neuronios e 90 camadas de entrada (X.shape[1] = 90)
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50, return_sequences = True)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50, return_sequences = True)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(LSTM(units = 50)) #camada oculta com 50 neuronios
regressor.add(Dropout(0.3)) #zerar 30% dos neuronios para evitar overfitting

regressor.add(Dense(units = 1, activation = 'linear')) #camada de saida com 1 neuronio

regressor.summary() #resumo da rede neural

In [14]:
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics= ['mean_absolute_error']) #compilar a rede neural com o otimizador rmsprop e a função de perda mean_squared_error (erro quadrático médio)

In [ ]:
regressor.fit(X, y, epochs = 100, batch_size = 32) #treinar a rede neural

In [54]:
base_teste = pd.read_csv('petr4_teste.csv') #base de teste
y_teste = base_teste.iloc[:, 1:2].values #variaveis independentes (preço de abertura)

In [55]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0) #concatenar as bases de treinamento e teste

In [56]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values #pegar os 90 valores anteriores ao primeiro valor da base de teste

In [57]:
entradas = entradas.reshape(-1, 1) #transformar em array

In [58]:
entradas = normalizador.transform(entradas) #normalizar os dados

In [59]:
X_teste = [] #variavel para armazenar os valores de teste
for i in range(90, 112): #pegar os 90 primeiros valores e colocar em X_teste
    X_teste.append(entradas[i-90:i, 0])

In [60]:
X_teste = np.array(X_teste) #transformar em array
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1)) #transformar em array tridimensional

In [ ]:
previsoes = regressor.predict(X_teste) #prever os valores

In [62]:
previsoes = normalizador.inverse_transform(previsoes) #desnormalizar os valores

In [ ]:
previsoes

In [ ]:
y_teste

In [ ]:
previsoes.mean() #média das previsões


In [ ]:
y_teste.mean() #média dos valores reais

In [ ]:
mean_absolute_error(y_teste, previsoes) #erro absoluto médio

In [ ]:
plt.plot(y_teste, color = 'red', label = 'Preço real') #plotar o gráfico com os valores reais
plt.plot(previsoes, color = 'blue', label = 'Previsões') #plotar o gráfico com as previsões
plt.legend() #mostrar a legenda